In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
from os.path import join
from web import for_website

from constants import COLUMNS, DATA_AGGREGATE_TYPES
from utils import (
    read_combined_labs_df, 
    read_combined_by_country_labs_df, 
    read_combined_by_site_labs_df,
    read_loinc_df,
    read_lab_meta_ci_df, 
    read_site_details_df,
    get_siteid_anonymous_map,
    get_anonymousid_color_maps,
    apply_theme
)

# Required Setups
- All combined datasets should be placed in `../data/combined` (e.g., `../data/combined/Labs-Combined{yymmdd}.csv`).
  - This jupyter notebook uses three data files:
     1. Site-level data (`Labs-CombinedBySite{yymmdd}.csv`)
     2. Country-level data (`Labs-CombinedByCountry{yymmdd}.csv`)
     3. Combined data (`Labs-Combined{yymmdd}.csv`)
- To save PNG files for visualizations, a folder named "output" should be present (i.e., `../output/`).

In [ ]:
# Titles
NUM_SITES = len(read_combined_by_site_labs_df()[COLUMNS.SITE_ID].unique())
DATA_DATE = "2020-04-10"
VIS_DATE = "2020-04-10"
SUBTITLE = f"Data as of {DATA_DATE} | {NUM_SITES} Sites | Plots generated on {VIS_DATE}"

# Path to save *.PNG files
SAVE_DIR = join("..", "output")

# Country Info
ALL_COUNTRY = "All countries"
ALL_COUNTRY_COLOR = "#444444"
COUNTRIES = [ "France", "Germany", "Italy", "Singapore", "USA" ]
COUNTRY_COLOR = [ "#0072B2", "#E69F00", "#009E73", "#CC79A7", "#D55E00" ]
COLOR_BY_COUNTRY = { COUNTRIES[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRIES)) }

# Data Preprocess

In [ ]:
def process_labs_df(df_lb):
    
    # Negative values to zeros
    df_lb.loc[df_lb[COLUMNS.NUM_PATIENTS] < 0, COLUMNS.NUM_PATIENTS] = 0
    df_lb.loc[df_lb[COLUMNS.MEAN_VALUE] < 0, COLUMNS.MEAN_VALUE] = 0
    df_lb.loc[df_lb[COLUMNS.STDEV_VAL] < 0, COLUMNS.STDEV_VAL] = 0

    # Upper and under bound for values
    df_lb["upper"] = df_lb[COLUMNS.MEAN_VALUE] + df_lb[COLUMNS.STDEV_VAL] 
    df_lb["under"] = df_lb[COLUMNS.MEAN_VALUE] - df_lb[COLUMNS.STDEV_VAL]
    df_lb["upper_p"] = df_lb[COLUMNS.NUM_PATIENTS] + df_lb[COLUMNS.MASKED_UPPER_BOUND_NUM_PATIENTS]
    df_lb["under_p"] = df_lb[COLUMNS.NUM_PATIENTS]
    df_lb[COLUMNS.NUM_PATIENTS] += (df_lb["upper_p"] - df_lb["under_p"]) / 2.0

    # Add readable names for LOINC
    loinc_df = read_loinc_df().set_index(COLUMNS.LOINC).rename(columns={'labTest': 'name'})
    df_lb["loinc_name"] = df_lb[COLUMNS.LOINC].apply(lambda code: 
        loinc_df.at[code, "name"] if loinc_df.at[code, "unit"] == "-1" else loinc_df.at[code, "name"] + " (" + loinc_df.at[code, "unit"] + ")"
    )

    # Number of sites
    df_lb[COLUMNS.NUM_SITES] = df_lb[COLUMNS.UNMASKED_SITES_NUM_PATIENTS]

    # Drop unused columns
    df_lb = df_lb.drop(columns=[
        COLUMNS.MASKED_UPPER_BOUND_NUM_PATIENTS,
        COLUMNS.UNMASKED_SITES_NUM_PATIENTS,
        COLUMNS.MASKED_SITES_NUM_PATIENTS
    ])
    
    # Remove data if no sites provided
    df_lb = df_lb[df_lb[COLUMNS.NUM_SITES] != 0]

    # Manual range for days since positive
    df_lb = df_lb[df_lb[COLUMNS.DAYS_SINCE_POSITIVE] >= -2]

    # Use more readable names
    df_lb.loc[df_lb[COLUMNS.SITE_ID] == "Combined", COLUMNS.SITE_ID] = ALL_COUNTRY

    return df_lb

# Read files
df_lb_country = read_combined_by_country_labs_df()
df_lb_country = process_labs_df(df_lb_country)
    
df_lb_combined = read_combined_labs_df()
df_lb_combined = process_labs_df(df_lb_combined)

df_lb_site = read_combined_by_site_labs_df() # Make sure to use anonymize() for site-level data (which adds hostpital data as well)
df_lb_site = process_labs_df(df_lb_site)

# Make sure to anonymize site-level data
hs_df = read_site_details_df().reset_index()
hs_df = hs_df[["Acronym", "Hospitals", "Country", "Country Color"]]

df_lb_site = df_lb_site.merge(hs_df, how="left", left_on=COLUMNS.SITE_ID, right_on="Acronym")
df_lb_site = df_lb_site.drop(columns={"Acronym"})
df_lb_site = df_lb_site.rename(columns={"Hospitals": COLUMNS.NUM_HOSPITALS})

siteid_anonymous_map = get_siteid_anonymous_map()
df_lb_site[COLUMNS.SITE_ID] = df_lb_site[COLUMNS.SITE_ID].apply(lambda x: siteid_anonymous_map[x])
ANONYMOUS_SITES = list(get_anonymousid_color_maps()[1].keys())
ANONYMOUS_COLORS = list(get_anonymousid_color_maps()[1].values())

# Merge all dfs
df_lb = pd.concat([df_lb_country, df_lb_combined])
df_lb = pd.concat([df_lb, df_lb_site])

# Set extent
NUM_PATIENTS_EXTENT = [0, max(df_lb[COLUMNS.NUM_PATIENTS])]
NUM_SITES_EXTENT = [0, max(df_lb[COLUMNS.NUM_SITES])]
DAYS_SINCE_EXTENT = [min(df_lb[COLUMNS.DAYS_SINCE_POSITIVE]), max(df_lb[COLUMNS.DAYS_SINCE_POSITIVE])]
    
df_lb

# Visualizations
## (Optional) Use pre-calculated information
- Make sure to put a "Lab_MetaCI_ran.csv" file below `data/labs/`

In [ ]:
ci_df = read_lab_meta_ci_df()
df_lb_combined = ci_df.rename(columns={
    "Lab": "loinc_name", 
    "mean": COLUMNS.MEAN_VALUE, 
    "se": COLUMNS.STDEV_VAL, 
    "ci_95L": "upper", 
    "ci_95U": "under", 
    "total_n": COLUMNS.NUM_PATIENTS
})

# We are using this as an alternative combined dataset
df_lb_combined[COLUMNS.SITE_ID] = ALL_COUNTRY

# TODO: To test ##########
# f_chart = alt.Chart(ci_df).transform_filter(
#     alt.datum["Lab"] == loinc
# )
# line = f_chart.mark_line().encode(
#     x="days_since_positive:Q",
#     y="mean"
# )

# band = line.mark_errorband().encode(
#     y="ci_95L",
#     y2="ci_95U"
# )

# (line + band)
##########################

# Change loinc names to the ones used in our data
consistent_loinc = {
    "alanine aminotransferase (ALT)": "Alanine aminotransferase (U/L)",
    "albumin": "Albumin (g/dL)",
    "aspartate aminotransferase (AST)": "Aspartate aminotransferase (U/L)",
    "total bilirubin": "Total bilirubin (mg/dL)",
    "C-reactive protein (CRP)": "C-reactive protein (mg/dL)",
    "creatinine": "Creatinine (mg/dL)",
    "lactate dehydrogenase (LDH)": "Lactate dehydrogenase (U/L)",
    "cardiac troponin": "Cardiac troponin (ng/mL)",
    "prothrombin time (PT)": "Procalcitonin (ng/mL)",
    "white blood cell count (Leukocytes)": "White blood cell count (10*3/uL)",
    "lymphocyte count": "Lymphocyte count (10*3/uL)",
    "neutrophil count": "Neutrophil count (10*3/uL)",
    "D-dimer": "D-dimer",
    "procalcitonin": "Prothrombin time (s)"
}
df_lb_combined["loinc_name"] = df_lb_combined["loinc_name"].apply(lambda x: consistent_loinc[x])
df_lb_combined.head()

## Lab values of interest

In [ ]:
loinc_of_interest = ["Creatinine (mg/dL)", "C-reactive protein (mg/dL)", "D-dimer", "Total bilirubin (mg/dL)", "White blood cell count (10*3/uL)"] # from LOINCS

domains = {
    "Creatinine (mg/dL)": {
        "y_domain_top": [0.5, 2.2],
        "y_domain_bottom": [0, 5.5]
    },
    "C-reactive protein (mg/dL)": {
        "y_domain_top": [20, 160],
        "y_domain_bottom": None
    },
    "D-dimer": {
        "y_domain_top": [1000, 7000],
        "y_domain_bottom": [0, 16000]
    },
    "Total bilirubin (mg/dL)": {
        "y_domain_top": [0.3,1.3],
        "y_domain_bottom": [0,5.5]
    },
    "White blood cell count (10*3/uL)": {
        "y_domain_top": [5, 15],
        "y_domain_bottom": None
    }
}

h = alt.hconcat()
for loinc in loinc_of_interest:

    no_axis_title = True if loinc != loinc_of_interest[0] else False
        
    site_level = lab_by_date(
        loinc=loinc, 
        data_level=DATA_AGGREGATE_TYPES.COMBINED_BY_SITE, 
        width=180, height=150, bar_size=3, no_axis_title=no_axis_title, show_stdev_top=True, no_line_top=True, no_line_bottom=False, legend_columns=12,
        y_domain_top=domains[loinc]["y_domain_top"], y_domain_bottom=domains[loinc]["y_domain_bottom"],
        is_num_hospitals=True, log_y_scale=False
    )

    h |= site_level
    
out = apply_theme(h, 
    legend_orient="bottom", 
    axis_title_font_size=10,
    label_font_size=12, 
    axis_label_font_size=10, 
    title_anchor="middle", 
    title_font_size=13
)

for_website(out, "Labs", "Five lab values by site") # TODO: Remove this before deploying notebook

out

## Lab values by country

In [ ]:
site_level_lab = lab_by_date(data_level=DATA_AGGREGATE_TYPES.COMBINED_BY_COUNTRY, show_stdev_top=True, no_line_top=True, height=200).properties(title={
    "text": "Lab values by country", 
    "subtitle": [SUBTITLE],
    "subtitleColor": "gray", 
    "dx": 60
})
site_level_lab = apply_theme(site_level_lab, legend_orient="right")

for_website(site_level_lab, "Labs", "Lab values by country") # TODO: Remove this before deploying notebook

site_level_lab

## Lab values by site

In [ ]:
site_level_lab = lab_by_date(data_level=DATA_AGGREGATE_TYPES.COMBINED_BY_SITE, show_stdev_top=True, no_line_top=True, height=200).properties(title={
    "text": "Lab values by site", 
    "subtitle": [SUBTITLE],
    "subtitleColor": "gray", 
    "dx": 60
})
site_level_lab = apply_theme(site_level_lab, legend_orient="right")

for_website(site_level_lab, "Labs", "Lab values by site") # TODO: Remove this before deploying notebook

site_level_lab